In [1]:
import ivy

In [2]:
class Regressor(ivy.Module):
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        super().__init__()

    def _build(self, *args, **kwargs):
        self.linear0 = ivy.Linear(self.input_dim, 128)
        self.linear1 = ivy.Linear(128, self.output_dim)

    def _forward(self, x):
        x = self.linear0(x)
        x = ivy.functional.relu(x)
        x = self.linear1(x)
        return x

In [3]:
ivy.set_backend('torch')  # set backend to PyTorch (or any other backend!)

<module 'ivy.functional.backends.torch' from '/home/ubuntu/ivy/ivy/functional/backends/torch/__init__.py'>

In [4]:
model = Regressor(input_dim=1, output_dim=1)
optimizer = ivy.Adam(0.3)

In [5]:
n_training_examples = 2000
noise = ivy.random.random_normal(shape=(n_training_examples, 1), mean=0, std=0.1)
x = ivy.linspace(-6, 3, n_training_examples).reshape((n_training_examples, 1))
y = 0.2 * x ** 2 + 0.5 * x + 0.1 + noise

In [6]:
def loss_fn(pred, target):
    return ivy.mean((pred - target) ** 2)

In [7]:
def predict(v, x):
    return model(x, v=v)

In [8]:
for epoch in range(40):
    # forward pass
    pred = model(x)
    loss = loss_fn(pred, y)
    loss_grads = ivy.grad(loss_fn, 0)(pred, y)

    pred, grads = ivy.execute_with_gradients(lambda params: predict(*params), (model.v, x), output_grads=loss_grads)

    # update parameters
    model.v = optimizer.step(model.v, grads)

    # print current loss
    print(f'Epoch: {epoch + 1:2d} --- Loss: {ivy.to_numpy(loss).item():.5f}')

print('Finished training!')

Epoch:  1 --- Loss: 2.51639
Epoch:  2 --- Loss: 12.46485
Epoch:  3 --- Loss: 18.43765
Epoch:  4 --- Loss: 6.79645
Epoch:  5 --- Loss: 2.08207
Epoch:  6 --- Loss: 4.37996
Epoch:  7 --- Loss: 0.94749
Epoch:  8 --- Loss: 4.93759
Epoch:  9 --- Loss: 1.29695
Epoch: 10 --- Loss: 1.28222
Epoch: 11 --- Loss: 2.65415
Epoch: 12 --- Loss: 0.86712
Epoch: 13 --- Loss: 1.76360
Epoch: 14 --- Loss: 1.49492
Epoch: 15 --- Loss: 0.18759
Epoch: 16 --- Loss: 1.29219
Epoch: 17 --- Loss: 1.18577
Epoch: 18 --- Loss: 0.34253
Epoch: 19 --- Loss: 0.75743
Epoch: 20 --- Loss: 0.64577
Epoch: 21 --- Loss: 0.18163
Epoch: 22 --- Loss: 0.49861
Epoch: 23 --- Loss: 0.61242
Epoch: 24 --- Loss: 0.17873
Epoch: 25 --- Loss: 0.10279
Epoch: 26 --- Loss: 0.41894
Epoch: 27 --- Loss: 0.35724
Epoch: 28 --- Loss: 0.08897
Epoch: 29 --- Loss: 0.12829
Epoch: 30 --- Loss: 0.27282
Epoch: 31 --- Loss: 0.20826
Epoch: 32 --- Loss: 0.08050
Epoch: 33 --- Loss: 0.09153
Epoch: 34 --- Loss: 0.14597
Epoch: 35 --- Loss: 0.10721
Epoch: 36 --- Loss